In [1]:
#Adapted from vendor code:  https://github.com/langchain-ai/langgraph/blob/main/examples/rag/langgraph_rag_agent_llama3_local.ipynb

## Loads text docs into Chroma vector store
## Catalog of document loaders is here:  https://python.langchain.com/v0.2/docs/integrations/document_loaders/

from langchain_community.document_loaders import PyPDFLoader
#from langchain_community.document_loaders import TextLoader
from chromadb.utils import embedding_functions
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_nomic.embeddings import NomicEmbeddings
#import chromadb
#from sentence_transformers import SentenceTransformer
from langchain_community.embeddings import SentenceTransformerEmbeddings
import gpt4all

#Set llm
local_llm = "llama3.1"

#ID directory to persist data
#client = chromadb.PersistentClient(path="/Users/reginaldstuart/Documents/IT/ML:AI/myRAG/chroma_db")

#create chroma_client and connect to server.  
#chroma_client = chromadb.HttpClient(host='localhost', port=8000)

#Select files we want to input
files = [
    "Flight_Training_Guide.pdf",
]

#Load files
docs = [PyPDFLoader(file).load() for file in files]
docs_list = [item for sublist in docs for item in sublist]

#Split files
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=100
)


#Here is the data we want to load
doc_splits = text_splitter.split_documents(docs_list)



# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="test_col_1",
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
#    embedding=embedding_functions.DefaultEmbeddingFunction(),
#    embedding=SentenceTransformerEmbeddings(),
    persist_directory="./ccbc_rag_db_2",
)

retriever = vectorstore.as_retriever()

##Now you should be able to read the vectorstore from another window



C:\Users\srstuartii\AppData\Local\Programs\Python\Python312\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (2016server). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


In [4]:
### Generate

from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate


# Prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
    Use ten sentences maximum <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question} 
    Context: {context} 
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

# prompt = PromptTemplate(
#     template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks. 
#     Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
#     Use three sentences maximum and keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
#     Question: {question} 
#     Context: {context} 
#     Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
#     input_variables=["question", "document"],
# )

llm = ChatOllama(model=local_llm, temperature=0)


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
question = "what types of aircraft can we get a license for?"
docs = retriever.invoke(question)
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)



C:\Users\srstuartii\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


To fly for a major airline, you typically need an Airline Transport Pilot (ATP) license. This is the highest level of licensure in the United States and is required by the Federal Aviation Administration (FAA). The ATP license requires a minimum of 1,500 hours of flight time, including 500 hours of cross-country flight time and 100 hours of night flight time. Additionally, you must be at least 23 years old and have a high school diploma or equivalent. You also need to pass a written exam and a practical test (checkride) with an FAA inspector.
